''' This program runs a dynamic topic model

#Date: June 27, 2020

#Author: Carly Knight

#https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb
#https://stackoverflow.com/questions/50413059/dynamic-topic-modeling-with-gensim-which-code'''


## set up

In [1]:
import logging
import os
from gensim import corpora, utils
from gensim.models.wrappers.dtmmodel import DtmModel
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary
from collections import defaultdict



### dtm

In [2]:
# you can also copy the path down directly. Change this variable to your DTM executable before running.
dtm_path = "/usr/local/bin/dtm-darwin64"

# Open corpus

In [3]:
file_loc = "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/future_texts_paragraphs/"
meta_loc = "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/future_texts/"

In [9]:
#read data
metadata = pd.read_csv(meta_loc + "metadata_futureperfect_paragraphs.csv") 

In [12]:
#limit to over 1930
metadata = metadata[(metadata['Year'] >= 1930)]

#create vars for filenames
metadata['document_filename'] = metadata['Filename'].str.replace(".xml", ".txt")
metadata['paragraph_filetime']= metadata['Filename'].str.replace(".xml", "") + "_P" + metadata['Paragraph'].astype(str) + ".txt"

#941727, 19
metadata.shape 

In [13]:
#pick relevant files
textfiles = [i for i in metadata['paragraph_filetime'].values.tolist()]

In [47]:
len(documentsdict)

830422

In [ ]:
# #open files (this step takes forever)
# for index, row in metadata.iterrows():
#     txtfile= row['paragraph_filetime']
#     #read in text file and turn into new variable
#     with open(file_loc + txtfile, 'r') as f:
#         metadata.loc[index,'text']= f.read()
#         f.close()

In [49]:
#empty dictionary
documentslist = []

#put txt files into dictionary
for txtfile in textfiles:
    with open(file_loc + txtfile, 'r') as f:
        documentslist.append(f.read())
        f.close()

In [25]:
#remove puncutation
#metadata['text'] = metadata['text'].str.replace(r'[^\w\s]+', '')

#remove numbers
#metadata['text'] = metadata['text'].str.replace(r'[\d]+', '')

#remove extraspaces
#metadata['text'] = metadata['text'].str.replace(r'\s+', ' ')

#turn to text
#documents = metadata['text'].tolist()

#turn dict to list
#documentslist = list(documentsdict.values())

## Preprocess

In [52]:
from gensim.parsing.preprocessing import preprocess_string

#processed corpus: removes punctuation, whitespaces, stopwords, stems, numbes
processed_corpus = [preprocess_string(document) for document in documentslist] 

#remove low frequency terms (terms with fewer than 20) (brings it down to 35,244 from 953,010 tokens)
frequency = defaultdict(int)

for text in processed_corpus:
    for token in text:
        frequency[token] += 1

processed_corpus = [
    [token for token in text if frequency[token] > 10]
    for text in processed_corpus
]

#dictionary
dictionary = Dictionary(processed_corpus)

#filter extremes: get rid of words that appear in no fewer than 5 documents
dictionary.filter_extremes(no_below=5)
len(dictionary) #43,025

#Bag of Words
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

In [55]:
#Time slice
counts = metadata[['Year', "paragraph_filetime"]].groupby(['Year']).agg(['count'])
timeslice=[i[0] for i in counts.values.tolist()]

In [58]:
#save
import pickle

f = open('/Users/carlyknight/Documents/Data/FuturePerfect/dynamic_topic_models/bow_corpus.pkl', 'wb') 
pickle.dump(bow_corpus, f , protocol=4)  

In [56]:
f = open('/Users/carlyknight/Documents/Data/FuturePerfect/dynamic_topic_models/timeslice.pkl', 'wb') 
pickle.dump(timeslice, f , protocol=4)  

In [57]:
f = open('/Users/carlyknight/Documents/Data/FuturePerfect/dynamic_topic_models/dictionary.pkl', 'wb') 
pickle.dump(dictionary, f , protocol=4)  

# DTM

In [59]:
model = DtmModel(dtm_path, bow_corpus, timeslice, num_topics=10,
                 id2word=dictionary, initialize_lda=True)

In [61]:
topics = model.show_topic(topicid=9, time=10, topn=10)
topics

[(0.0292026838633479, 'product'),
 (0.017882360518980095, 'year'),
 (0.016729942254754424, 'new'),
 (0.014984024016644724, 'increas'),
 (0.014765016519143985, 'sale'),
 (0.01308772828994461, 'market'),
 (0.011498421128094634, 'plant'),
 (0.010815669325945602, 'oper'),
 (0.009202203094732327, 'continu'),
 (0.009196817038446653, 'compani')]

In [19]:
#https://towardsdatascience.com/topic-modeling-with-gensim-a5609cefccc

# save

In [62]:
f = open('/Users/carlyknight/Documents/Data/FuturePerfect/dynamic_topic_models/topic_para_10.pkl', 'wb') 
pickle.dump(model, f , protocol=4)          
f.close()                 